Привет! Меня зовут Валерий Гриднев, я провожу ревью твоей работы. Предлагаю общаться на «ты», если не против.

В работе ты увидишь комментарии, помеченные разным цветом.

<div class="alert alert-success">
Зеленым цветом помечены оброзцовые решения. Которые можно использовать в дальнейшем.
</div>

<div class="alert alert-warning">
Желтым цветом отмечено то, что ты можешь улучшить или доработать. Можешь учесть это при выполнении следующих работ или доработать проект сейчас, но это не обязательно.
</div>

<div class="alert alert-danger">
Красным цветом помечены моменты, которые необходимо доработать, чтобы проект был принят. Уверен, что у тебя все получится. Буду ждать твою работу на повторное ревью. 
</div>

Будет комфортнее работать над проектом вместе, если ты будешь **писать об изменениях** в проекте по моим рекомендациям. Для этого выбери для своих комментариев 

<div class="alert alert-block alert-info">заметный цвет</div>

Пожалуйста, **не перемещай, не изменяй и не удаляй мои комментарии**. Всё это поможет выполнить повторную проверку твоего проекта оперативнее.
___

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

`сalls` — количество звонков,

`minutes` — суммарная длительность звонков в минутах,

`messages` — количество sms-сообщений,

`mb_used` — израсходованный интернет-трафик в Мб,

`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

Импортируем библиотеки:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier

Импортируем данные:

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Смотрим общую информацию о данных:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


In [5]:
print('Число дубликатов:', df.duplicated().sum())

Число дубликатов: 0


Пропусков и дубликатов в данных нет, в изменении типа данных нет необходимости. Можно приступить к анализу данных.

<div class="alert alert-success">
    
**Комментарий ревьюера**
    
Отличное начало)Приступим к самому интересному)
</div>

## Разбейте данные на выборки

Разделим данные на признаки и цель:

In [6]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Сформируем тренировочную, тестовую и валидационную выборки в соотношении 3:1:1:

In [7]:
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=.4, random_state=1234)
features_test, features_valid, target_test, target_valid = train_test_split(features_temp, target_temp, test_size=.5, random_state=12345)

In [8]:
print(f'Размер тренировочной выборки: {features_train.shape[0]}')
print(f'Размер тестовой выборки: {features_test.shape[0]}')
print(f'Размер валидационной выборки: {features_valid.shape[0]}')

Размер тренировочной выборки: 1928
Размер тестовой выборки: 643
Размер валидационной выборки: 643


Мы подготовили данные к обучению и тестированию. 

<div class="alert alert-success">
    
**Комментарий ревьюера**
    
Отлично)
</div>

## Исследуйте модели

Поскольку мы будем исследовать несколько моделий, то напишем универсальную функцию, которая будет принимать название модели и дипазон гиперпараметров. Гиперпараметры будем подбирать с помощью `GridSearchCV`, а формирование валидационой выборки осуществлять с помощью `StratifiedKFold`.

<div class="alert alert-warning">
    
**Комментарий ревьюера**
    
Круто, что ты знаешь такие методы и умеешь их применять. Но зачем в таком случае ты делил выборку на 3 части?

Это просто введение)Не нужны усложнять себе жизнь)
</div>

In [9]:
def best_parameters(model, parameter_grid):
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    grid_search = GridSearchCV(model, parameter_grid, n_jobs=-1, cv=kfold)
    grid_result = grid_search.fit(features_train, target_train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Определим оптимальные гиперпараметры для:

`DecisionTreeClassifier`

In [10]:
best_parameters(DecisionTreeClassifier(random_state=1234), {'criterion': ['entropy', 'gini'],'max_depth': range(2, 50, 2)})

Best: 0.771266 using {'criterion': 'entropy', 'max_depth': 4}


`RandomForestClassifier`

In [11]:
best_parameters(RandomForestClassifier(random_state=1234), {'criterion': ['entropy', 'gini'],'max_depth': range(1, 20, 1),'n_estimators': range(2, 20, 2)})

Best: 0.792012 using {'criterion': 'gini', 'max_depth': 7, 'n_estimators': 18}


`KNeighborsClassifier`

In [12]:
best_parameters(KNeighborsClassifier(), {'n_neighbors': range(2, 20, 2)})

Best: 0.746369 using {'n_neighbors': 8}


`CatBoostClassifier`

In [ ]:
best_parameters(CatBoostClassifier(verbose=False, random_state=1234), {"iterations": range(5, 100, 5)})

`MLPClassifier`

In [ ]:
best_parameters(MLPClassifier(random_state=1234), {"hidden_layer_sizes": range(5, 20, 5),
                                                "activation": ["identity", "logistic", "tanh", "relu"], 
                                                "solver": ["lbfgs", "sgd", "adam"],
                                               "max_iter": [1000, 5000]})

Оптимальные гиперпараметры определены.

Можем приступить к отбору моделий по валидационной ваборке. Напишем функцию, принимающую название модели и ее оптимальные гиперпараметры:

In [ ]:
def model_valid(model):
    model.fit(features_train,target_train)
    predictions=model.predict(features_valid)
    print('accuracy:',accuracy_score(predictions,target_valid))

Проверим

`DecisionTreeClassifier`


In [ ]:
model_valid(DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=1234))

`RandomForestClassifier`

In [ ]:
model_valid(RandomForestClassifier(criterion='gini', max_depth=7, n_estimators=18, random_state=1234))

`KNeighborsClassifier`

In [ ]:
model_valid(KNeighborsClassifier(n_neighbors=8))

`CatBoostClassifier`

In [ ]:
model_valid(CatBoostClassifier(verbose=False, iterations=35, random_state=1234))

`MLPClassifier`

In [ ]:
model_valid(MLPClassifier(activation='tanh', hidden_layer_sizes=15, solver='lbfgs', max_iter=1000, random_state=1234))

Модели: `DecisionTreeClassifier`, `RandomForestClassifier`, `KNeighborsClassifier`, `CatBoostClassifier` имеют **accuracy** больше 0,75, а модель `MLPClassifier` - меньше, исключим ее. Вероятнее всего, модель `MLPClassifier` имеет столь низкую **accuracy**, поскольку модели, основанные на нейронных сетях, требуют нормирования и центрирования данных перед анализом.

## Проверьте модель на тестовой выборке

Проверим модели, имеющие **accuracy** больше 0,75 на тестовых данных. Напишем для этого функцию:

In [ ]:
def model_test(model):
    model.fit(features_train,target_train)
    predictions = model.predict(features_test)
    accuracy = accuracy_score(target_test, predictions)
    print('Accuracy: {:.3f}'.format(accuracy))

In [ ]:
model_test(RandomForestClassifier(criterion='gini', max_depth=7, n_estimators=18, random_state=1234))

In [ ]:
model_test(CatBoostClassifier(verbose=False, iterations=35, random_state=1234))

In [ ]:
model_test(DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=1234))

In [ ]:
model_test(KNeighborsClassifier(n_neighbors=8))

Мы проанализировали 5 моделей и определили для них оптималньые гиперпараметры. Модель `RandomForestClassifier` проявила себя лучше всего, а `MLPClassifier` показала неудовлетворительные результаты, вызванные, вероятно, недостаточной подготовкой данных.

<div class="alert alert-success">
    
**Комментарий ревьюера**
    
Евгений)Ты молодец, что знаешь больше чем дают на данном этапе, но использовать их везде просто так это плохо. Чтобы получить нужные результаты здесь ничего сложного не нужно)Проект принимаю)Успехов в дальнейшем обучении)
    
P.S.Если есть какие-то вопросы, пожалуйста задай их через куратора)
</div>

## (бонус) Проверьте модели на адекватность

Посмотрим соотношение двух вариантов в исходных данных:

In [ ]:
df['is_ultra'].value_counts(normalize=True)

Получается, что если бы модель всегда предсказывала только вариант **0**, то ее **accuracy** составляла всего **0,694**, что ниже всех полученных нами значений.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
